In [ ]:
# import lsst.summit.extras

In [ ]:
# lsst.summit.__file__

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from lsst.summit.extras.plotting.psfPlotting import makeAzElPlot, makeTableFromSourceCatalogs, makeFocalPlanePlot, makeEquatorialPlot
from lsst.obs.lsst import LsstComCam, LsstCam
import matplotlib.pyplot as plt

In [ ]:
from lsst.daf.butler import Butler
import numpy as np

## set up

In [ ]:
import sys
# sys.path.append("../../notebooks/")
sys.path.append("/sdf/data/rubin/shared/image_quality/imsim/notebooks/")                #file with convertion between seqnum and collection
from collection_dictionary_shared import collection_dictionary

In [ ]:
folder = '/sdf/data/rubin/shared/image_quality/imsim/'
#Define the butler data configuration and collection (una tantum )
config = folder+'repo'
seqnum_base = 5023071800000
collection_dict = collection_dictionary()

# sequence number

In [ ]:
seqnum = 155
visitid_complete = seqnum+seqnum_base
collections = collection_dict[visitid_complete]
collections

In [ ]:
collection_dict[visitid_complete]

In [ ]:
butler = Butler(config, collections=collections)

In [ ]:
# butler.collections.query_info(collections[0], include_summary=True)

In [ ]:
registry = butler.registry

In [ ]:
# for dt in sorted(registry.queryDatasetTypes()):
#     print(dt)

In [ ]:
postisr = list(registry.queryDatasets('icSrc', collections=collections, detector=90))
postisr
#exposure_ids = np.array([data.dataId['exposure'] for data in postisr])
# exposure_ids

In [ ]:
icSrcs = {}
visit_id=postisr[0].dataId['visit']
for i in range(189):
    try:
        icSrcs[i] = butler.get("icSrc", dataId={"visit":visit_id, "detector":i})
    except:
        continue
visitInfo = butler.get("postISRCCD.visitInfo", dataId={"exposure":visit_id, "detector":94})
table = makeTableFromSourceCatalogs(icSrcs, visitInfo)

In [ ]:
# icSrcs[90]

In [ ]:
table.meta['rotTelPos']*180/np.pi

In [ ]:
table.meta['rotSkyPos']*180/np.pi

In [ ]:
camera = LsstCam().getCamera()

## alt/az in mm

In [ ]:
fig, axes = plt.subplots(nrows=2, ncols=2, figsize=(10, 9), sharex=True, sharey=True)
makeAzElPlot(fig, axes, table, camera)
plt.show()

In [ ]:
# fig, axes = plt.subplots(nrows=2, ncols=2, figsize=(10, 9), sharex=True, sharey=True)
# makeFocalPlanePlot(fig, axes, table, camera)
# plt.show()

In [ ]:
# fig, axes = plt.subplots(nrows=2, ncols=2, figsize=(10, 9), sharex=True, sharey=True)
# makeEquatorialPlot(fig, axes, table, camera)
# plt.show()

# Convert mm to field angle and write output file

In [ ]:
from ellipticity_util import addFieldCoords_to_Table, makeOCSPlot, read_batoid_table
import numpy as np

In [ ]:
table = addFieldCoords_to_Table( table, camera, table.meta['aaRot'] )

In [ ]:
fig, axes = plt.subplots(nrows=2, ncols=2, figsize=(10, 9), sharex=True, sharey=True)
makeOCSPlot(fig, axes, table, camera)
plt.show()

In [ ]:
from astropy.io import ascii

In [ ]:
outname = folder + 'rzs/ellip_maps/imsim_seq%.8d.csv'%seqnum
# outname = '/home/r/rzanmar/coords_seq%.8d.csv'%seqnum

In [ ]:
outname

In [ ]:
ascii.write(table[ 'aa_field_x','aa_field_y','aa_Ixx','aa_Iyy','aa_Ixy', ],outname, format='csv', fast_writer=False, overwrite=True)

# verify mm to field [deg] conversion

In [ ]:
hx, hy = table['x'] * 0.2 / 36, table['y'] * 0.2 / 36      # 0.2 arcsec / pix * pix / 10um -> 0.2/36 deg/mm

In [ ]:
hx_rot, hy_rot = table['aa_x'] * 0.2 / 36, table['aa_y'] * 0.2 / 36

In [ ]:
fig, axes = plt.subplots(nrows=1, ncols=1, figsize=(10, 9), sharex=True, sharey=True)
plt.plot( np.sqrt( table['aa_field_x']**2+table['aa_field_y']**2), np.sqrt( (table['aa_field_x'] - hx_rot)**2 + (table['aa_field_y'] - hy_rot)**2)*3600, '.' )
axes.set_xlabel('field radius [deg]')
axes.set_ylabel('difference [arcsec]')
axes.set_title('FIELD Vs. linear')
plt.show()